In [1]:
import re
from collections import defaultdict
import json
import os
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [2]:
#sci-fi popularity
link = 'https://www.imdb.com/search/title/?genres=sci-fi&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=3396781f-d87f-4fac-8694-c56ce6f490fe&pf_rd_r=PD077K5BS0CH9730H7YS&pf_rd_s=center-1&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr1_i_2'



In [3]:
cwd = os.getcwd()
image_folder = os.path.join(cwd, 'images')

In [9]:
if not os.path.exists(image_folder):
    os.mkdir(image_folder)

In [5]:
# get an url for eacg image
resp = requests.get(link)
data = resp.content
filtered = BeautifulSoup(data).find_all('div', attrs = {'class': 'lister-item mode-advanced'})



In [6]:
names = []
image_urls = []
ratings = []
years = []
info_dict = defaultdict(list)
for i in filtered:
    image_url = i.select('img')[0]['loadlate']
    year_x = i.find('span', attrs = {'class': 'lister-item-year text-muted unbold'}).text
    year = int(re.findall('\d+', year_x)[0])
    #sometimes, there aren't ratings so I replace them with -1
    try:
        rating = float(i.find('div', attrs = {'class': 'inline-block ratings-imdb-rating'}).select('strong')[0].text)
    except:
        rating = -1  
    name = i.select('img')[0]['alt']
    names.append(name)
    image_urls.append(image_url)
    years.append(year)
    ratings.append(rating)
    info_dict[name] = [{'url': image_url, 'year': year, 'rating': rating}]

In [7]:
# store image info into json file
with open('./info.json', 'w+') as fw:
    json.dump(info_dict, fw)

In [10]:
#download the images to local image_folder by urls
for i in range(len(names)):
    store_as = os.path.join(image_folder, names[i])+'.jpg'
    img_data = requests.get(image_urls[i]).content
    with open(store_as, 'wb') as fb:
        fb.write(img_data)